In [3]:
from ztp import ztpoisson
import sqlite3
import math
from collections import Counter
import numpy as np

In [4]:
mu = 1.3
dbpath = '../WebServer2/participants.db'
# 获取总的参与人数
sqlGetCount =  'select count(*) from participants'
sqlUpdateGroupID = ''
con = sqlite3.connect(dbpath)

cur = con.cursor()
cur.execute(sqlGetCount)
record = cur.fetchone()
con.close()

totalNumber = 9#record[0]
print(totalNumber)

9


In [5]:
# 各种size的群组所占百分比，服从ZTP分布
groupPercents = [ztpoisson.rvs(mu) for _ in range(100)]
# print(groupPercents)
c = Counter(groupPercents)
results = [tup[1] for tup in c.most_common()][:5]
print(results) # 各种size的群组所占百分比，这里没考虑人数

[44, 32, 17, 5, 1]


In [6]:
# 计算人数
peopleActually = [results[i]*(i+1) for i in range(len(results))]
peopleActually = np.array(peopleActually)
sumPeople = peopleActually.sum() # 需要的总人数
peoplePercents = peopleActually/sumPeople # 各组按照人数计算，所占的百分比
print(peoplePercents)

# 实际人数
peopleNumber = np.round(totalNumber*peoplePercents) # 计算各组人数
print(peopleNumber)
# 计算结果表示，groupsize为index的组有value个人

[ 0.23913043  0.34782609  0.27717391  0.10869565  0.02717391]
[ 2.  3.  2.  1.  0.]


In [7]:
# reassignment fro each group
# 如果一个组分不完的话，或者一个组不够分的话，归还到pool中，分配到其他组；
pool = totalNumber - int(sum(peopleNumber))
peopleNumber = peopleNumber.astype(int).tolist()
peopleNumber.reverse()
print(peopleNumber)
print(pool)

[0, 1, 2, 3, 2]
1


In [8]:
result = []

for idx,num in enumerate(peopleNumber):
    # get from pool
    tempNum = peopleNumber[idx] + pool
    groupIdx = len(peopleNumber) - idx
    result.append(tempNum // groupIdx)
    remains = tempNum % groupIdx
    # return to pool
    pool = remains
    
result.reverse()
print(result)
print('NO. of people each group: ')
numInEachGroup = [gno*(idx+1) for idx,gno in enumerate(result)]
print(numInEachGroup)
print(sum(numInEachGroup))

# 把群组配置输入URL：119.23.128.14:9934/reassignGroup?groupConf=2,2,1,0,0

[2, 2, 1, 0, 0]
NO. of people each group: 
[2, 4, 3, 0, 0]
9


In [10]:
def groupFromConfig(groupConf):
    numInGroups = [gno * (idx + 1) for idx, gno in enumerate(groupConf)]
    groupIDs = []
    leaders = []
    groupSizes = []
    
    # 生成单人组，id从17开始，leader都是1
    startID = 17
    for i in range(numInGroups[0]):
        leaders.append(1)
        groupSizes.append(1)
        groupIDs.append(startID)
        startID += 1

    # 多人组，id从1开始，每组第一个人是leader
    startID = 0
    for idx,no in enumerate(numInGroups[1:]):
        for i in range(no):
            if i % (idx+2) == 0: # 每组第一个人
                startID += 1
                leaders.append(1)
            else:
                leaders.append(0)
            groupIDs.append(startID)    
            groupSizes.append(idx+2)
    
    return groupIDs,groupSizes,leaders

print(groupFromConfig([2, 2, 1, 0]))


([17, 18, 1, 1, 2, 2, 3, 3, 3], [1, 1, 2, 2, 2, 2, 3, 3, 3], [1, 1, 1, 0, 1, 0, 1, 0, 0])
